In [2]:
import csv
import io
import numpy as np
import sklearn as sk
from matplotlib import pyplot as plt

%matplotlib inline

In [6]:
col_data = [("Genre", "|S20"),
            ("Artist", "|S50"),
            ("Track", "|S50"),
            ("Track ID", "|S22"),
            ("Popularity", int),
            ("Acousticness", float),
            ("Danceability", float),
            ("Duration (MS)", int),
            ("Energy", float),
            ("Instrumentalness", float),
            ("Key", "|S2"),
            ("Liveness", float),
            ("Loudness", float),
            ("Mode", "|S5"),
            ("Speechiness", float),
            ("Tempo", float),
            ("Time Signature", "|S3"),
            ("Valence", float)]

with io.open("SpotifyFeatures.csv", "r", encoding="utf-8") as f:
    data = np.genfromtxt(f, delimiter=",", skip_header=1, comments=None, dtype=col_data)

x array - 230253 x 10  
acousticness 5  
danceability 6  
duration 7  
energy 8  
instrumentalness 9  
liveness 11  
loudness 12  
speechiness 14  
tempo 15  
valence 17  

should we also include key, time signature, mode? (will need to map to numbers)  

y array - 230253 x 1  
genres (the true labels) mapped to numbers

In [ ]:
x = np.empty((230253, 10))
y = np.empty((230253, 1))
genre_mapping_dict = {}
genre_idx = 0

for i in range(len(data)):
    row = data[i]
    genre = row[0].decode("utf-8")
    if not genre in genre_mapping_dict:
        genre_mapping_dict[genre] = genre_idx
        genre_idx += 1
    y[i] = genre_mapping_dict[genre]
    
    values = [row[5], row[6], row[7], 
              row[8], row[9], row[11], 
              row[12], row[14], row[15], row[17]]
    x[i, :] = values